# Data Extraction

### This Notebook focuses on extracting the information about the restaurants in Jaipur. The process is divided into 3 Phases:  

#### [Phase - 1](#phase1): 
Extract the links for each locality from ZOMATO.
#### [Phase - 2](#phase2): 
Extract the information about each Restaurant from a locality.
#### [Phase - 3](#phase3): 
Geocode the address of a restaurant into latitude and longitude using Google Geocoding API.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import googlemaps
import json

# Phase - 1 <a name='phase1'></a>
**Get the links to all the localities from Zomato**

In [2]:
url = 'https://www.zomato.com/jaipur/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

**We will fetch the names of localities along with their respective urls and store it in a dictionary.**

In [3]:
list_ = soup.find_all('section')[2]
data = dict()
for i in list_.find_all('a'):
    link = i.attrs['href']
    locality = i.contents[0].string.strip("\n ")
    data.update({locality: link})

In [4]:
for key in data:
    print(data[key])

https://www.zomato.com/jaipur/c-scheme-restaurants
https://www.zomato.com/jaipur/malviya-nagar-restaurants
https://www.zomato.com/jaipur/tonk-road-restaurants
https://www.zomato.com/jaipur/vaishali-nagar-restaurants
https://www.zomato.com/jaipur/mi-road-restaurants
https://www.zomato.com/jaipur/mansarovar-restaurants
https://www.zomato.com/jaipur/raja-park-restaurants
https://www.zomato.com/jaipur/bani-park-restaurants
https://www.zomato.com/jaipur/bais-godam-restaurants
https://www.zomato.com/jaipur/adarsh-nagar-restaurants
https://www.zomato.com/jaipur/ajmer-highway-restaurants
https://www.zomato.com/jaipur/lal-kothi-restaurants
https://www.zomato.com/jaipur/gopalbari-restaurants
https://www.zomato.com/jaipur/sodala-restaurants
https://www.zomato.com/jaipur/shyam-nagar-restaurants
https://www.zomato.com/jaipur/sindhi-camp-restaurants
https://www.zomato.com/jaipur/pink-city-restaurants
https://www.zomato.com/jaipur/amer-restaurants
https://www.zomato.com/jaipur/civil-lines-restaurants

# Phase - 2 <a name='phase2'></a>
**Get the details about all the restaurants in a locality.**

Restaurant Names and their Category

In [5]:
def getName(soup):
    resInfo = soup.find_all('article', class_='search-result')
    name = [item.find(class_='result-title') for item in resInfo]
    name = [item.string.strip("\n ") for item in name]
    category = [item.find_all('a',class_='zdark') for item in resInfo]
    for index, item in enumerate(category):
        for innerIndex, val in enumerate(item):
            category[index][innerIndex] = val.string
    return name, category

Restaurant Address

In [6]:
def getAddress(soup):
    resAdd = soup.find_all('div', class_='search-result-address')
    address = []
    for i in resAdd:
        address.append(i.string.strip(" "))
    return address

Restaurant Ratings and Votes

In [7]:
def getRateAndVotes(soup):
    rate = soup.find_all('div', class_='search_result_rating')
    ratings = []
    votes = []
    for r in rate:
        try:
            val = r.find_all('div', class_='rating-popup')[0].string.strip("\n ")
            ratings.append(val)
            vote = r.find_all('span')[0].string.strip(" votes")
            votes.append(vote)
        except:
            votes.append('No-Votes')
    return ratings, votes

Cuisines for each restaurant and Cost for TWO.

In [8]:
def getCostAndCuisine(soup):
    cuiCost = soup.find_all('div', class_='search-page-text')
    cost = []
    cuisines = []
    for c in cuiCost:
        temp = []
        try:
            for item in c.find(class_='clearfix').find_all('a'):
                temp.append(item.string.strip("\n "))
            cuisines.append(temp)
            cost.append((c.find(class_='res-cost').find(class_='pl0').string.strip(" ")))
        except:
            cost.append('No Cost given.')
    return cost, cuisines

Driver Function to extract all the data using the above functions

In [9]:
def extractData(headers,loc,key):
    urlPhase2 = loc
    key = [key]
    while True:
        responsePhase2 = requests.get(urlPhase2, headers=headers)
        soup = BeautifulSoup(responsePhase2.text, 'html.parser')
        
        tempNames,tempCat = getName(soup)
        names.append(tempNames)
        category.append(tempCat)
        
        address.append(getAddress(soup))
        
        tempRatings, tempVotes = getRateAndVotes(soup)
        ratings.append(tempRatings)
        votes.append(tempVotes)
        
        tempCost, tempCuisines = getCostAndCuisine(soup)
        costForTwo.append(tempCost)
        cuisines.append(tempCuisines)
        
        locality.append(key*len(tempNames))
        
#       Look for the link of next page and print it.
        links = soup.find('div', class_='search-pagination-top')
        print(urlPhase2)
        try:
            nextPage = links.find('a', class_='next').attrs['href']
            urlPhase2 = 'https://www.zomato.com'+nextPage
        except: # if the next page does not exists, break out of the loop.
            break

Now that we are all set for extraction, let's define the variables to store the data.  

In [10]:
names=[]
address=[]
category=[]
votes=[]
ratings=[]
costForTwo=[]
cuisines=[]
locality=[]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
for key in data:
    url = data[key]+'?nearby=0'
    extractData(headers,url,key)
    print(key)

https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=2
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=3
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=4
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=5
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=6
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=7
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=8
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=9
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=10
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=11
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=12
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=13
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page=14
https://www.zomato.com/jaipur/c-scheme-restaurants?nearby=0&page

KeyboardInterrupt: 

The results are list of lists, to proceed further, we must flatten them out

In [ ]:
import numpy as np
import itertools

locality_ = list(itertools.chain(*locality))
names_ = list(itertools.chain(*names))
address_ = list(itertools.chain(*address))
category_ = list(itertools.chain(*category))
costForTwo_ = list(itertools.chain(*costForTwo))
cuisines_ = list(itertools.chain(*cuisines))
votes_ = list(itertools.chain(*votes))
ratings_ = list(itertools.chain(*ratings))

Storing the data into Pandas DataFrame and exporting to a csv file.

In [ ]:
df = pd.DataFrame(data=[locality_,names_,address_,category_,
                        costForTwo_,cuisines_,ratings_,votes_])
df = df.T
columns = ['Locality','RestaurantName','Address',
           'Category','CostForTwo','Cuisines','Ratings','votes']
df.columns = columns

In [ ]:
df.to_csv('Restaurants.csv', index=False)

# Phase - 3 <a name="phase3">

Import the restaurants data.

In [2]:
res = pd.read_csv('Restaurants.csv')

In [3]:
res

,Locality,RestaurantName,Address,Category,CostForTwo,Cuisines,Ratings,votes
0,C Scheme,Kanha,"E 62, Bhagat Singh Marg, C Scheme, Jaipur","['Sweet Shop', 'Quick Bites']",₹500,"['North Indian', 'South Indian', 'Fast Food', ...",4.2,995
1,C Scheme,Stepout Cafe,"P 14, Sehdev Marg, Ashok Nagar, C Scheme, Jaipur","['Café', 'Casual Dining']",₹800,"['Cafe', 'Italian', 'Lebanese', 'Mexican', 'Co...",4.5,1385
2,C Scheme,Burger Farm,"3 & 4, Trimurty, V-Jai City Point Building, Ah...",['Quick Bites'],₹400,"['Burger', 'Fast Food', 'Beverages']",4.1,1427
3,C Scheme,Domino's Pizza,"C 18, Silver Square Mall, Bhagwandas Marg, C S...",['Quick Bites'],₹700,"['Pizza', 'Fast Food']",3.3,1258
4,C Scheme,Thali and More,"C-46 B, Sarojini Marg, Above Punjab National B...",['Casual Dining'],"₹1,100","['North Indian', 'South Indian', 'Chinese', 'F...",4.4,2339
...,...,...,...,...,...,...,...,...
4433,Khatipura Road,The KDS Food Point,"Shop No 28, 29, Teeja Nagar, Panchyawala, Sirs...",['Quick Bites'],₹250,"['Pizza', 'Fast Food']",NaN,No-Votes
4434,Khatipura Road,Diva Tiffin Center,"House D-88, Shanti Nagar, Khatipura Road, Jaipur",[],₹200,['North Indian'],NaN,No-Votes
4435,Khatipura Road,Shamrock,"Hotel Shamrock Inn, Sirsi Road, Panchyawala, N...",['Casual Dining'],₹800,['North Indian'],NaN,No-Votes
4436,Khatipura Road,The Royal Thikaana,"50-51, Chand Bihari Nagar, Near Shani Mandir, ...",['Quick Bites'],₹350,"['Maharashtrian', 'North Indian', 'Fast Food']",NaN,No-Votes


Drop the records with no address

In [4]:
res.drop(res.loc[res['Address']=='Delivery Only'].index, inplace=True)
res.reset_index(drop=True, inplace=True)

Create a list of Address and Names of the Restaurants

In [24]:
address = [item for item in res['Address']]
name = [item for item in res['RestaurantName']]

Import the Google Map's Geocoding API Key

In [7]:
API_KEY = None
with open('/home/hotpie/Projects/API_KEYS/API_KEYS.json','r') as keyFile:
    f = json.load(keyFile)
    
API_KEY = f['google-api-key']
gmaps = googlemaps.Client(key=API_KEY)

Now, we will define a function to geocode the street address of the restaurants.
The address will be reformated to be name+address of the restaurant.  
eg:   
Restaurant: The Royal Thikana  
Address: 50-51, Chand Bihari Nagar, Near Shani Mandir, Khatipura Road, Jaipur  
This info will be used as:  
The Royal Thikaana, 50-51, Chand Bihari Nagar, Near Shani Mandir, Khatipura Road, Jaipur
  
    
If the geographical coordinates for an address cannot be found, the function will return [0,0] as the values for latitude and longitude.

In [8]:
def getLatitudeLongitude(add, name):
    try:
        address = name+', '+add
        g = gmaps.geocode('{}, Rajasthan'.format(address))
        if len(g)==0:
            return [0,0]
        lat_lng = [g[0]['geometry']['location']['lat'], g[0]['geometry']['location']['lng']]
        print('Address: {}, Lat: {}, Long: {}'.format(
            address,
            lat_lng[0],
            lat_lng[1]))
    except:
        g = gmaps.geocode('{}, Rajasthan'.format(add))
        lat_lng = [g[0]['geometry']['location']['lat'], g[0]['geometry']['location']['lng']]
        print('Address: {}, Lat: {}, Long: {}'.format(
            add,
            lat_lng[0],
            lat_lng[1]))
    return lat_lng

In [26]:
latlng=[]

In [29]:
for add, name in zip(address, name):
    latlng.append(getLatitudeLongitude(add, name))

Address: Kanha, E 62, Bhagat Singh Marg, C Scheme, Jaipur, Lat: 26.9122583, Long: 75.8005896
Address: Stepout Cafe, P 14, Sehdev Marg, Ashok Nagar, C Scheme, Jaipur, Lat: 26.9058994, Long: 75.8020779
Address: Burger Farm, 3 & 4, Trimurty, V-Jai City Point Building, Ahinsa Circle, Near Vodafone Store, C Scheme, Jaipur, Lat: 26.9142461, Long: 75.8049298
Address: Domino's Pizza, C 18, Silver Square Mall, Bhagwandas Marg, C Scheme, Jaipur, Lat: 26.9150431, Long: 75.8099258
Address: Thali and More, C-46 B, Sarojini Marg, Above Punjab National Bank, C Scheme, Jaipur, Lat: 26.910926, Long: 75.80567900000001
Address: Little Italy, C 11, KK Square, C Scheme, Jaipur, Lat: 26.9108494, Long: 75.7986025
Address: Tapri Central, B4 E, 3rd Floor, Surana Jewellers, Opposite Central Park, C Scheme, Jaipur, Lat: 26.905601, Long: 75.811182
Address: The Yellow House - The Robot Restaurant, Silver Square Mall, Near Rajmandir Cinema, C Scheme, Jaipur, Lat: 26.9152097, Long: 75.8098025
Address: La Palma Cafe 

Address: House Of Bunkins-HOBS, A/4B, 2nd Floor, Chittora House, Opposite Udyog Bhawan, Tilak Marg, C Scheme, Jaipur, Lat: 26.9013308, Long: 75.7970227
Address: My Froyoland, G-5A, City Mall, Bhagwan Das Road, C Scheme, Jaipur, Lat: 26.9143122, Long: 75.8082551
Address: Fabcafe By Fabindia, GT Galleria, Ashok Road, Ashok Nagar, C Scheme, Jaipur, Lat: 26.9142593, Long: 75.8069021
Address: The Polo Bar - Rambagh Palace, Rambagh Palace, Bhawani Singh Road, C Scheme, Jaipur, Lat: 26.89836, Long: 75.808433
Address: Keventers - The Original Milkshake, Plot 25, Yudhister Marg, Near Deer Park, C Scheme, Jaipur, Lat: 26.9040848, Long: 75.7970953
Address: Bharat Bits N Bites, J 4, Ramesh Marg, Subhash Marg Road, C Scheme, Jaipur, Lat: 26.9063811, Long: 75.7963704
Address: Gangaur Sweets, J32, Subhash Marg, Near Bagadia Bhawan, C Scheme, Jaipur, Lat: 26.9076255, Long: 75.7988088
Address: Jaipur Down Town, Plot O-10, Oyo Townhouse, Ashok Marg, Opposite Green House, Ahinsa Circle, C Scheme, Jaipur,

Address: Charis Cafe, 2nd Floor, 18 Mission Compound, Near Yamaha Motors, Panch Batti, C Scheme, Jaipur, Lat: 26.9132822, Long: 75.7940182
Address: Uncle Coffee, Near Income Tax Office, Statchu Circle, C Scheme, Jaipur, Lat: 26.9079896, Long: 75.8041935
Address: Kitchen Daddy Restro, D 49, Chomu House, C Scheme, Jaipur, Lat: 26.821418, Long: 75.8481472
Address: Chai Paani, 7, Vijay City Point, Ahinsa Circle, Ashok Marg, C Scheme, Jaipur, Lat: 26.9145807, Long: 75.8058844
Address: Saini's Kachoriwala, Subhash Marg Corner, Bagadiya Bhawan, C Scheme, Jaipur, Lat: 26.9088953, Long: 75.80011209999999
Address: Big Daddy's, C 84, Ground Floor, Prithviraj Road, C Scheme, Jaipur, Lat: 26.9084777, Long: 75.8032149
Address: Khansama The Traditional Kitchen, P 14, Sehdev Marg, C Scheme, Jaipur, Lat: 26.9077383, Long: 75.8060598
Address: The Crunch, Plot F-1-A, Ramesh Marg, Opposite Masjid, C Scheme, Jaipur, Lat: 26.9057583, Long: 75.7946798
Address: Pandit Ji Ki Famous Pav Bhaji And Bhelpuri, Shop

Address: Thali Express, Near Udyog Bhawan Opposite Bhawani Singh Road, C Scheme, Jaipur, Lat: 26.9014328, Long: 75.7991621
Address: Kanha, Plot 30-31-32, Kalyan Colony, Opposite Gaurav Towers, Malviya Nagar, Jaipur, Lat: 26.8545153, Long: 75.80823459999999
Address: Domino's Pizza, Ground Floor, GT Central, Malviya Nagar, Jaipur, Lat: 26.8562753, Long: 75.8060016
Address: T2 Dhabha, 10/KB, Krishna Puri, Malviya Nagar, Jaipur, Lat: 26.8453287, Long: 75.8082065
Address: GVD Soni's Cafe, Shop 2, Shopping Centre, Mahesh Marg, D-Block, Malviya Nagar, Jaipur, Lat: 26.8532022, Long: 75.8067812
Address: Midnight Vegas, 11/01, 4th Floor, Girdhar Marg, Malviya Nagar, Jaipur, Lat: 26.8433712, Long: 75.8146179
Address: Hotel Mini Mahal, 2, Durga Vatika, Near Trimurti Appartment, Model Town, Malviya Nagar, Jaipur, Lat: 26.8447563, Long: 75.8215607
Address: Burger King, Shop 3-4, Ground Floor, GT Central Mall, Malviya Nagar, Jaipur, Lat: 26.8562523, Long: 75.8060959
Address: Sethi Tikka Kabab Curry, 

Address: Chicken Chatkara, Shop 41A, Sector 9, Palika Bazar, Malviya Nagar, Jaipur, Lat: 26.8498597, Long: 75.8093444
Address: Burger Farm Express, FC/03B & 03C, Ground Floor, Gaurav Tower, Podium, Malviya Nagar, Jaipur, Lat: 26.8557197, Long: 75.8069337
Address: Sky Corner, B 17, Mahalaxmi Nagar, Near World Trade Park, Malviya Nagar, Jaipur, Lat: 26.8517888, Long: 75.80519869999999
Address: 3D Restro Lounge, 8th, 9th Floor, Coorporate Tower, Jawahar Circle, Sector 8, Malviya Nagar, Jaipur, Lat: 26.843529, Long: 75.802357
Address: Stallion By Green Apple, 9th Floor, SDC Building, Pradhan Marg Corner, Calgeri Road, Malviya Nagar, Jaipur, Lat: 26.8589205, Long: 75.80910829999999
Address: Bhukaddd, C 28, Malviya Nagar, Jaipur, Lat: 26.8478744, Long: 75.82141469999999
Address: Tan-Sukh, Opposite Gaurav Towers, Malviya Nagar, Jaipur, Lat: 26.8545198, Long: 75.8082343
Address: Cafe Hungry Birds, B-6, Mahalaxmi Nagar, Behind WTP, Malviya Nagar, Jaipur, Lat: 26.8528005, Long: 75.8054418
Addres

...

Address: The Thar Restro And Cafe, 1, 200 Feet Bypass Road, Shanti Path, Jhotwara, Jaipur, Lat: 26.951338, Long: 75.733076
Address: Khandelwal Dhaba, Kardhani Thana, Kalwar Road, Govindpura, Jhotwara, Jaipur, Lat: 26.9547345, Long: 75.77892279999999
Address: Chowdhary Pavitra Bhojnalaya And Restaurant, 28, 80 Feet Road, Laxmi Nagar III, Jagannath Puri, Kaanta Chauraha, Suraj Nagar, Jhotwara, Jaipur, Lat: 26.9446142, Long: 75.7436445
Address: Sahu Ji Ki Special Kota Kachori, Near Panchayat Samiti, Jhotwara, Jaipur, Lat: 26.9431536, Long: 75.7572806
Address: Club Elaaka Restro & Bar, Plot 1A/121, A -II, Sanjay Nagar 1, Kalwar Road, Jhotwara, Jaipur, Lat: 26.9406556, Long: 75.7461348
Address: Sai Chaap Soya, 8 Sanjay Nagar B Kalwar Road, Jhotwara, Jaipur, Lat: 26.867071, Long: 75.764076
Address: Sugar Hut Bakery & Cafe, B-66, Gotra Plaza, Shri Ram Nagar, Khirni Phatak Road, Jhotwara, Jaipur, Lat: 26.9381504, Long: 75.7432383
Address: Pink Pasta, 80/81, Opposite Power House, Lata Nagar, Ma

Address: Bombay Bakers, Shop B-4, Abhinandan Plaza, Satya Nagar, Jhotwara, Jaipur, Lat: 26.9399983, Long: 75.7441219
Address: Jungle House Cafe, Joshi Marg, Opposite Maxx Hospital, Kalwar Road, Jhotwara, Jaipur, Lat: 26.9452859, Long: 75.73567849999999
Address: Haunted House Cafe, Plot 12, Kalyan Kunj, Kirani Phatak Road, Jhotwara, Jaipur, Lat: 26.9434169, Long: 75.74350760000002
Address: Bhati Gravy And Grill, 36, Boring Road, Ward 7, Jagannathpuri, Jhotwara, Jaipur, Lat: 26.8667864, Long: 75.76958119999999
Address: Mehai Fast Food, Plot A-259, Tara Nagar, Near Reliance Fresh, Khirni Phatak, Jhotwara, Jaipur, Lat: 26.93506, Long: 75.73924070000001
Address: Yash Fast Food And Restaurant, Plot 09 B, Near Kanta Choraha, Jhotwara, Jaipur, Lat: 26.9443465, Long: 75.7437486
Address: City Confectionary, 301 A, 80 Feet Road, Jaganath Puri, Near Kaanta Kalwar Road, Jhotwara, Jaipur, Lat: 26.9545943, Long: 75.7455944
Address: Sharma Pavitra Bhojnalaya, 1A, Vijay Singh Pathik Nagar, 200 Feet Byp

Address: Royal Chick, Shop 1A/3, Hari Nagar, Lanka Puri Mod, Opposite SBI Bank, Shastri Nagar, Jaipur, Lat: 26.9516707, Long: 75.7969656
Address: Vadilal Hangout, E-84, Near Science Park, Shastri Nagar, Jaipur, Lat: 26.941531, Long: 75.797213
Address: Friends Restro, G1, Krishna Square, Subash Nagar, Shastri Nagar, Jaipur, Lat: 26.9362881, Long: 75.79469379999999
Address: The Bawarchi, 4, Shopping Centre, Subhash Nagar, Near Allen Institute, Shastri Nagar, Jaipur, Lat: 26.936213, Long: 75.795925
Address: Spicy King, A 17, Ram Nagar Colony, Shastri Nagar, Jaipur, Lat: 26.9473362, Long: 75.79757719999999
Address: Cream Bell, H-59A, Sanjay Colony, RPA Road, Shastri Nagar, Lat: 26.949131, Long: 75.78935729999999
Address: Ridhkaran Mishthan Bhandar, G 59 & 60, Shubhash Nagar Shopping Centre, Shastri Nagar, Jaipur, Lat: 26.9366524, Long: 75.7952931
Address: Fresh Fresh Breakfast, 155, Gate 3, Sindhu Nagar, Nahari Ka Naka, Power House, Shastri Nagar, Jaipur, Lat: 26.9465143, Long: 75.8049647


Address: Cafe Amer - The Gold Palace & Resorts, The Gold Palace & Resorts, 8th Mile Amer Fort, Jaipur-Delhi Highway, Kukas, Jaipur, Lat: 27.0455072, Long: 75.9037307
Address: Gulab Ji Sweets & Restaurant, Rose Arcad Sai Vatika, Near Sai Temple, Kukas, Jaipur, Lat: 27.0391343, Long: 75.8919925
Address: Sethi's Family Restaurant, Gole Market, Near Petrol Pump Road, Jawahar Nagar, Jaipur, Lat: 26.8933312, Long: 75.8392329
Address: Tea Connect Cafe, D-33, Shanti Path, Patrakar Colony, Jawahar Nagar, Jaipur, Lat: 26.8921203, Long: 75.83255919999999
Address: Serving Spoon, D 63, Shantipath, Jawahar Nagar, Jaipur, Lat: 26.8949979, Long: 75.8358183
Address: Crazy Coffee, 5-JHA-16A, Sector 5, Jawahar Nagar, Jaipur, Lat: 26.8878801, Long: 75.8295558
Address: The Food Barn, CC-199, Gole Market, Near Petrol Pump Road, Jawahar Nagar, Jaipur, Lat: 26.8938056, Long: 75.8378534
Address: Buffet House Restaurant, 5 TA 19, Sher Singh Chauraha, Jawahar Nagar, Jaipur, Lat: 26.889211, Long: 75.831226
Addres

Address: Pizza Square, Plot A-2 & A-3, Govardhanbadi Scheme, Khatipura Road, Jaipur, Lat: 26.9223094, Long: 75.7221472
Address: Sejal Cakes N Bake, Ground Floor, National Super Market, Rajendra Nagar, Sirsi Road, Khatipura Road, Jaipur, Lat: 26.9143148, Long: 75.7379423
Address: ChocoHeart, Hanuman Nagar Extension, Amar Nagar, Khatipura Road, Jaipur, Lat: 26.9205224, Long: 75.7348964
Address: Tamsa's Kitchen, Plot 21-29, Kishanpura House, Kishanpura Marg, Marudhar Vihar, Khatipura Road, Jaipur, Lat: 26.9244168, Long: 75.74772639999999
Address: The Food 4 U, G-1, Plot 131, Krishan Path, Rajendra Nagar, Sirsi Road, Near Khatipura Road, Jaipur, Lat: 26.9203415, Long: 75.7333553
Address: Gulab Sweets, Jharkhand Tiraha, Khatipura Road, Jaipur, Lat: 26.926225, Long: 75.748419
Address: Mugal Restaurant, 54, Shri Ram Market, Khatipura Road, Jaipur, Lat: 26.9409046, Long: 75.761707
Address: On The Way, Near Alfa Academy, Jaswant Nagar, Khatipura Puliya, Khatipura Road, Jaipur, Lat: 26.9304145, 

In [42]:
df = pd.DataFrame(latlng)
df.to_csv('coordinates.csv')
res['Latitude'] = df[0]
res['Longitude'] = df[1]

Let's drop all the records with no geographical coordinates

In [37]:
res.drop(res.loc[res['Latitude']==0].index, inplace=True)

In [39]:
res.reset_index(drop=True, inplace=True)

In [41]:
res.to_csv('RestaurantGeocoded.csv', index=False)